## 8DM40: Assignment Week 1 (Group 5):
# Two simple machine learning models

Initialization:

In [1]:
import sys
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes, load_breast_cancer
import math
import matplotlib.mlab as mlab

# add subfolder that contains all the function implementations
# to the system path so we can import them
sys.path.append('code/')

# Get the diabetes data
diabetes = load_diabetes()

# Get the breast cancer data
breast_cancer = load_breast_cancer()

***Question*** (in text): Why we need to use the `np.newaxis` expression in the examples above? 

***Answer:*** If you do not use np.newaxis:

In [2]:
X = diabetes.data[:,3]
print(X.shape)

(442,)


You get a vector with shape (442,) instead of an array with shape (442,1)

## Exercises + Answers

### Linear Regression

Implement training and evaluation of a linear regression model on the diabetes dataset using only matrix multiplication, inversion and transpose operations. Report the mean squared error of the model.

To get you started we have implemented the first part of this exercise (fitting of the model) as an example.

#### Answer:

In [3]:
# The actual implementation is in linear_regression.py,
from linear_regression import *

# Load the dataset, split in training and testing set
X_train = diabetes.data[:300, :]
y_train = diabetes.target[:300, np.newaxis]
X_test = diabetes.data[300:, :]
y_test = diabetes.target[300:, np.newaxis]

# Determine the parameters from the training data with the least squared method
w_hat = lsq(X_train, y_train)

# Print the parameters
print('Fitted parameters:\n', w_hat)

# Determine mean squared error (mse)
mean_sq_error = mse(X_test, y_test, w_hat)
print('\nMean squared error: ', mean_sq_error)

Fitted parameters:
 [[ 152.34786452]
 [ -16.57607993]
 [-254.66532396]
 [ 560.98630022]
 [ 278.91811152]
 [-393.41357305]
 [  97.05460405]
 [ -19.0023093 ]
 [ 169.46450327]
 [ 632.95050374]
 [ 114.21638941]]

Mean squared error:  2794.5690145007284


### Weighted linear regression

Assume that in the dataset that you use to train a linear regression model, there are identical versions of some samples. This problem can be reformulated to a weighted linear regression problem where the matrices $\boldsymbol{\mathrm{X}}$ and $\boldsymbol{\mathrm{Y}}$ (or the vector $\boldsymbol{\mathrm{y}}$ if there is only a single target/output variable) contain only the unique data samples, and a vector $\boldsymbol{\mathrm{d}}$ is introduced that gives more weight to samples that appear multiple times in the original dataset (for example, the sample that appears 3 times has a corresponding weight of 3). 

<p><font color='#770a0a'>Derive the expression for the least-squares solution of a weighted linear regression model (note that in addition to the matrices $\boldsymbol{\mathrm{X}}$ and $\boldsymbol{\mathrm{Y}}$, the solution should include a vector of weights $\boldsymbol{\mathrm{d}}$).</font></p>

#### Answer:
Notation:\
Vector: $\vec{a}$\
Tensor: $\boldsymbol{\mathrm{A}}$

For weighted linear regression, the sum of squares is:
\begin{equation}
{RSS}_{w}(\vec{w}) = \sum_{i=1}^{N}{d_i(y_i- \vec{x_i}^T \vec{w} )}
\end{equation}
Each squared error gets multiplied by a corresponding weight instead of having multiple instances in the input-output data where the same error would be added to the sum multiple times.

In order to convert to tensor notation, we need to be able to multiple element-wise, say $\vec{a} * \vec{b} = \vec{c}$, where $\vec{c}$ is the same shape as the other 2 vectors, and $c_i = a_i b_i$ for element $i$. All the vectors have the same length. Note that $\vec{a} * \vec{b} = \vec{b} * \vec{a}$.

For tensors, say $\boldsymbol{\mathrm{A}} * \vec{b} = \boldsymbol{\mathrm{C}}$, the dimensions need to be $\boldsymbol{\mathrm{A}} \in R^{m \times n}; \vec{b} \in R^{m \times 1}; \boldsymbol{\mathrm{C}} \in R^{m \times n}$, i.e. the resulting tensor has the same dimensions as the original, and the length of the vector $\vec{b}$ needs to be the same as the number of rows of $\boldsymbol{\mathrm{A}}$ (or in the case of $\boldsymbol{\mathrm{A}} * \vec{b}^T$, the same number of columns). Note that here also, $\boldsymbol{\mathrm{A}} * \vec{b} = \vec{b} * \boldsymbol{\mathrm{A}}$. 

Using this notation, the sum of squares becomes:
\begin{equation}
{RSS}_{w}(\vec{w}) = (\vec{y}-\vec{X} \vec{w})^T (\vec{y}-\vec{X} \vec{w}) * \vec{d}
\end{equation}

With $\vec{d},\vec{y} \in R^{N \times 1}; \vec{w} \in R^{p \times 1}$ and $\vec{X} \in R^{N \times p}$.

Differentiate ${RSS}_{w} (\vec{w})$ to $\vec{w}$:

${RSS}_{w}(\vec{w}) = (\vec{y}-\boldsymbol{\mathrm{X}} \vec{w})^T (\vec{y} * \vec{d} - \boldsymbol{\mathrm{X}} \vec{w} * \vec{d})$

$\frac{\partial {RSS}_{w}}{\partial \vec{w}} = -\boldsymbol{\mathrm{X}}^T (\vec{y} * \vec{d} - \boldsymbol{\mathrm{X}} \vec{w} * \vec{d}) + (\vec{y} -\boldsymbol{\mathrm{X}} \vec{w})^T (- \boldsymbol{\mathrm{X}} * \vec{d})$

$\frac{\partial {RSS}_{w}}{\partial \vec{w}} = -\boldsymbol{\mathrm{X}}^T (\vec{y} - \boldsymbol{\mathrm{X}} \vec{w}) * \vec{d} - (\boldsymbol{\mathrm{X}} * \vec{d})^T (\vec{y} -\boldsymbol{\mathrm{X}} \vec{w}) $

On the first term, we need to apply $\boldsymbol{\mathrm{A}}^T \cdot \vec{b} * \vec{c} = (\boldsymbol{\mathrm{A}} * \vec{c})^T \cdot \vec{b}$, with $\boldsymbol{\mathrm{A}} \in R^{m \times n}; \vec{b} \in R^{m \times 1}; \vec{c} \in R^{m \times 1}$. This follows from dimension analysis (${n \times m} \cdot {m \times 1} * {m \times 1} = {n \times 1} = ({n \times m} * {1 \times m}) \cdot { m\times 1}$).

Apply to $\frac{\partial {RSS}_{w}}{\partial \vec{w}}$:

$\frac{\partial {RSS}_{w}}{\partial \vec{w}} = -(\boldsymbol{\mathrm{X}} * \vec{d})^T (\vec{y} - \boldsymbol{\mathrm{X}} \vec{w}) - (\boldsymbol{\mathrm{X}} * \vec{d})^T (\vec{y} - \boldsymbol{\mathrm{X}} \vec{w}) $

Result of differentiation of ${RSS}_{w}$ to $\vec{w}$:
\begin{equation}
\frac{\partial {RSS}_{w}}{\partial \vec{w}} = -2(\boldsymbol{\mathrm{X}} * \vec{d})^T (\vec{y} - \boldsymbol{\mathrm{X}} \vec{w}) 
\end{equation}

Solve for $\vec{w}$:

$(\boldsymbol{\mathrm{X}} * \vec{d})^T (\vec{y} - \boldsymbol{\mathrm{X}} \vec{w}) = \vec{0}$

$ (\boldsymbol{\mathrm{X}} * \vec{d})^T \vec{y} - (\boldsymbol{\mathrm{X}} * \vec{d})^T \boldsymbol{\mathrm{X}} \vec{w} = \vec{0}$

$(\boldsymbol{\mathrm{X}} * \vec{d})^T \boldsymbol{\mathrm{X}} \vec{w} = (\boldsymbol{\mathrm{X}} * \vec{d})^T \vec{y}$

With as result for weighted linear regression:
\begin{equation}
\vec{w} = ((\boldsymbol{\mathrm{X}} * \vec{d})^T \boldsymbol{\mathrm{X}})^{-1} (\boldsymbol{\mathrm{X}} * \vec{d})^T \vec{y}
\end{equation}

Check result with dimension analysis:

${p \times 1} = (({N \times p} * {N \times 1})^T \cdot {N \times p})^{-1} \cdot ({N \times p} * {N \times 1})^T \cdot {N \times 1}$

${p \times 1} = ({p \times N} \cdot {N \times p})^{-1} \cdot {p \times N} \cdot {N \times 1}$

${p \times 1} = ({p \times p})^{-1} \cdot {p \times 1}$

Note that the matrix to be inverted is square, as it needs to be.

Check result if unweighted/normal: For the unweighted case, $\vec{d}$ is a vector of ones. Then $\boldsymbol{\mathrm{X}} * \vec{d} = \boldsymbol{\mathrm{X}}$. Indeed, substituting this gives the unweighted solution: $\vec{w} = (\boldsymbol{\mathrm{X}}^T \boldsymbol{\mathrm{X}})^{-1} \boldsymbol{\mathrm{X}}^T \vec{y}$.

### $k$-NN classification

Implement a $k$-Nearest neighbors classifier from scratch in Python using only basic matrix operations with `numpy` and `scipy`. Train and evaluate the classifier on the breast cancer dataset, using all features. Show the performance of the classifier for different values of $k$ (plot the results in a graph). Note that for optimal results, you should normalize the features (e.g. to the $[0, 1]$ range or to have a zero mean and unit standard deviation).

#### Answer:

In [1]:
from normalisation import *
from k_nn_classification import *
from visualisation import *

#load data
X = breast_cancer.data
#normalize data
X = normalize(X)

#divide dataset in train and test sets
X_train = X[:350]
y_train = breast_cancer.target[:350, np.newaxis]
X_test = X[350:]
y_test = breast_cancer.target[350:, np.newaxis]

k = list(range(1,11)) #assign different values for k 
correct = [] 

#perform model on all values for k, calculate the overlap with the prediction and the  test target vector
for i in k: 
    acc = accuracy(X_train, X_test, y_train, y_test, i)
    correct.append(acc)

#visualize the accuracy for each value of k 
visualisation(correct,k)


ModuleNotFoundError: No module named 'normalisation'

### $k$-NN regression

Modify the $k$-NN implementation to do regression instead of classification. Compare the performance of the linear regression model and the $k$-NN regression model on the diabetes dataset for different values of $k$.

#### Answer:
[???]

### Class-conditional probability

Compute and visualize the class-conditional probability (conditional probability where the class label is the conditional variable, i.e. $P(X = x \mid Y = y)$ for all features in the breast cancer dataset. Assume a Gaussian distribution.

<p><font color='#770a0a'>Based on visual analysis of the plots, which individual feature can best discriminate between the two classes? Motivate your answer.</font></p>

#### Answer:

In [2]:
from sep_class import * 
from statistics import * 

#Step 1: Load the data.
breast_cancer = load_breast_cancer()
#Step 2: split the dataset into a training and testing set and operate on each feature seperately per loop.  
for i in range(len(breast_cancer.data[0])):  
    X_train = breast_cancer.data[:300, np.newaxis, i]
    y_train = breast_cancer.target[:300]
    X_test =  breast_cancer.data[300:, np.newaxis, i]
    y_test =  breast_cancer.target[300:, np.newaxis]

    #Step 3: Seperate the targets (0 or 1) for the feature.
    seperated_classes = sep_class(X_train, y_train) #This will return 2 lists within a list with each list correseponding to either class 0 or 1
    list_class_0 = seperated_classes[0]
    list_class_1 = seperated_classes[1]
    
    #Step 4: Determine the mean and standard deviation of the feature per class.
    mean_class_0 = mean(list_class_0)
    std_class_0 = std(list_class_0)

    mean_class_1 = mean(list_class_1)
    std_class_1 = std(list_class_1)

    #Step 5: plot the histograms per class per feature with corresponding normal distribution
    x_class_0 = np.linspace(min(list_class_0), max(list_class_0), 100)
    plt.hist(list_class_0, bins="auto",normed=True, alpha = 0.8, color= 'red', label='Class 0')
    plt.plot(x_class_0, mlab.normpdf(x_class_0, mean_class_0, std_class_0), color='black')
    
    x_class_1 = np.linspace(min(list_class_1), max(list_class_1), 100)
    plt.hist(list_class_1, bins="auto",normed=True,alpha = 0.8, color='green', label='Class 1')
    plt.plot(x_class_1, mlab.normpdf(x_class_1, mean_class_1, std_class_1), color = 'black')
    
    pyplot.legend(loc='upper right')
    pyplot.title('Feature ' + str(i+1))
    plt.show()

ModuleNotFoundError: No module named 'sep_class'